# Import libraries

In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

# Load prepared dataframe

In [53]:
df = pd.read_csv('../data/df_prepped.csv')
pd.set_option('display.max_columns', None)

print('df.shape:', df.shape)
df.head()

df.shape: (32359, 85)


,Year,Countries,Sand_1,Sand_2,Sand_3,Sand_4,Sand_5,Sand_6,Sand_7,Clay_1,Clay_2,Clay_3,Clay_4,Clay_5,Clay_6,Clay_7,OC_1,OC_2,OC_3,OC_4,OC_5,OC_6,OC_7,PAW_1,PAW_2,PAW_3,PAW_4,PAW_5,PAW_6,PAW_7,pcp_1,pcp_2,pcp_3,pcp_4,pcp_5,pcp_6,pcp_7,pcp_8,pcp_9,pcp_10,pcp_11,pcp_12,tmax_1,tmax_2,tmax_3,tmax_4,tmax_5,tmax_6,tmax_7,tmax_8,tmax_9,tmax_10,tmax_11,tmax_12,tmin_1,tmin_2,tmin_3,tmin_4,tmin_5,tmin_6,tmin_7,tmin_8,tmin_9,tmin_10,tmin_11,tmin_12,spi_12mon_1,spi_12mon_2,spi_12mon_3,spi_12mon_4,spi_12mon_5,spi_12mon_6,spi_12mon_7,spi_12mon_8,spi_12mon_9,spi_12mon_10,spi_12mon_11,spi_12mon_12,Y_maize_major,Farm,Sow_Maize_month_int,Harvest_Maize_month_int,maize_lag-1,maize_lag-2,maize_lag-3
0,2007,Angola,50,51,51,48,45,46,46,37,35,36,39,42,42,42,0.52,0.23,0.17,0.09,0.04,0.02,0.02,0.15,0.15,0.14,0.13,0.10,0.07,0.07,193.640232,120.569706,227.203369,293.985844,37.067890,3.281593,5.613327,15.461922,28.531075,77.929020,256.519079,110.747576,301.279337,301.643356,302.561828,301.998669,303.294018,301.273750,301.828465,300.877661,301.054315,301.439985,301.087249,299.965674,293.385797,293.903355,293.715671,294.182027,293.914887,289.529794,289.196482,290.720474,291.646143,292.901044,293.562402,292.396252,0.673668,0.751071,0.918140,0.939547,0.958888,0.961378,0.968957,0.982076,1.012476,0.994139,1.051325,0.891661,0.615357,104_Angola,9,4,0.554392,0.721607,0.620005
1,2007,Angola,62,64,63,59,58,59,59,27,25,26,29,31,30,30,0.11,0.05,0.07,0.04,0.02,0.02,0.01,0.11,0.10,0.10,0.09,0.07,0.07,0.03,89.417934,94.982147,124.716759,16.971111,0.288486,0.000000,0.000000,0.000000,0.000000,1.527309,26.616097,1.892090,303.652199,303.925860,301.532011,303.486865,303.480002,300.632807,300.447617,303.846624,307.456561,307.836796,306.928968,307.922920,290.917679,290.372625,291.020000,290.329827,288.060277,283.087506,283.774803,285.841064,288.790434,289.599520,290.990366,289.516585,1.335389,1.495891,1.225347,0.842463,0.839905,0.840094,0.840128,0.840204,0.755807,0.269691,-0.364644,-0.698008,0.257656,99_Angola,9,4,0.117051,0.300217,0.212699
2,2007,Angola,69,71,70,67,65,65,66,19,16,18,21,24,24,23,0.09,0.06,0.07,0.04,0.02,0.02,0.02,0.10,0.10,0.10,0.09,0.07,0.07,0.07,65.143585,45.137405,95.773697,23.603439,0.114441,0.000000,0.000000,0.000000,0.000000,2.087116,15.358448,1.543999,307.124048,307.082463,304.947372,305.374424,304.348148,300.925227,300.947202,304.486048,308.383310,308.862071,308.818092,308.833916,293.565072,292.685367,292.796358,292.373439,289.358169,284.500931,284.923087,286.443955,290.339812,291.690485,293.005088,292.176617,1.154411,0.623367,0.227251,-0.108796,-0.107750,-0.107804,-0.107810,-0.108608,-0.140493,-0.574688,-1.053915,-1.099187,4.286831,108_Angola,9,4,3.093239,4.044452,2.295351
3,2007,Angola,60,63,61,57,53,53,53,29,26,28,32,35,36,36,0.46,0.16,0.14,0.08,0.05,0.04,0.03,0.12,0.13,0.12,0.12,0.11,0.10,0.09,206.006050,200.949192,296.508312,166.140556,1.461506,0.158787,0.019550,5.464077,85.021496,330.712795,381.351948,136.991739,296.459724,297.477400,298.498063,299.143915,300.855134,299.695964,300.559739,302.105577,301.508277,298.939023,297.489408,296.588843,288.633720,288.680392,288.648945,289.158368,287.485643,284.321525,284.031431,285.352713,287.550441,288.532437,288.667658,288.051526,0.506923,0.595885,0.698705,0.340710,0.321866,0.322036,0.322048,0.314241,0.414349,0.433416,0.436678,0.300900,0.700384,102_Angola,9,4,0.677797,0.907431,0.783018
4,2007,Angola,67,69,68,63,61,61,61,22,19,21,25,28,28,29,0.15,0.09,0.09,0.05,0.02,0.01,0.01,0.11,0.11,0.11,0.11,0.08,0.04,0.04,170.792580,142.048359,155.379295,48.344135,0.401497,0.006199,0.000477,0.025272,0.078201,19.632339,109.449863,18.647194,300.344314,301.227051,301.143868,303.398120,305.024706,303.274907,304.376005,306.934593,308.368511,307.674898,304.787832,305.405160,292.201118,292.184213,292.205253,291.979049,289.903756,286.127356,287.130153,287.468311,289.656432,291.560132,292.873329,291.132666,1.518292,1.823893,1.623040,1.435289,1.433381,1.433460,1.433452,1.432657,1.415767,1.139544,0.957729,0.125110,0.553450,43_Angol

In [54]:
df.Countries.nunique()

30

In [55]:
df.Farm.nunique()

3887

# Train-test split

In [56]:
# Dropping countries and farms because one-hot-encoding would make 4000+ rows
# We tested that and found that it did not improve performance but increased runtime
df_label = df.loc[:,['Countries','Farm']]
df = df.drop(['Countries','Farm'], axis=1)

In [57]:
# Separate a test set, the year 2016
test = df[df.Year == 2016]
df_train = df[df.Year < 2016]

In [58]:
print('The training set has years: ', list(df_train.Year.unique()))
print('The test set has years: ', list(test.Year.unique()))

The training set has years:  [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
The test set has years:  [2016]


# Baseline selection using CV and years 2007-2015

In [59]:
# Cross-validation set-up
# https://stackoverflow.com/questions/58069691/how-to-create-a-train-test-split-of-time-series-data-by-year

year_list = df_train['Year'].unique().tolist()
splits = {'train': [], 'val': []}

for idx, yr in enumerate(year_list[:-1]):
    if yr < 2010:
        # To get only the last 5 splits
        continue
    train_yr = year_list[:idx+1]
    test_yr = [year_list[idx+1]]
    print('TRAIN: ', train_yr, 'VAL: ',test_yr)
    
    splits['train'].append(df_train.loc[df_train.Year.isin(train_yr), :])
    splits['val'].append(df_train.loc[df_train.Year.isin(test_yr), :])

TRAIN:  [2007, 2008, 2009, 2010] VAL:  [2011]
TRAIN:  [2007, 2008, 2009, 2010, 2011] VAL:  [2012]
TRAIN:  [2007, 2008, 2009, 2010, 2011, 2012] VAL:  [2013]
TRAIN:  [2007, 2008, 2009, 2010, 2011, 2012, 2013] VAL:  [2014]
TRAIN:  [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014] VAL:  [2015]


In [60]:
SEED = 1

In [61]:
regressors = {
    "KNN": KNeighborsRegressor(n_neighbors=5), # 1
    "Random Forest": RandomForestRegressor(n_estimators=50, random_state=SEED), # 2
    "Gradient Boosting": AdaBoostRegressor(n_estimators=50, random_state=SEED) # 3
}

In [62]:
# This will take about 15 minutes

reg_names = []
cv_MAE = []
cv_RMSE = []
cv_folds = []
train_years = []
val_year = []

i = 1
for train, val in zip(splits['train'], splits['val']):

    print('Fold: ', i)

    # Shuffle
    train = train.sample(frac=1, random_state=SEED)
    val = val.sample(frac=1, random_state=SEED)
    
    # X and y
    X_train = train.drop(columns=['Y_maize_major','Year'], axis=1)
    y_train = train['Y_maize_major']
    X_val = val.drop(columns=['Y_maize_major','Year'], axis=1)
    y_val = val['Y_maize_major']

    # Scale to [0,1] range
    sc = MinMaxScaler()
    X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
    X_val = pd.DataFrame(sc.transform(X_val), columns=X_val.columns)

    # Fit and predict
    for reg_name, reg in regressors.items():
        reg_names.append(reg_name)
        reg.fit(X_train, y_train)
        y_val_pred = reg.predict(X_val)
        rmse = mean_squared_error(y_val, y_val_pred, squared=False)
        mae = mean_absolute_error(y_val, y_val_pred)
        cv_MAE.append(mae)
        cv_RMSE.append(rmse)
        cv_folds.append(i)
        train_years.append(list(train.Year.unique()))
        val_year.append(list(val.Year.unique()))

        print(reg_name)
        print('RMSE:', rmse)
        print('MAE:', mae)

    i += 1
    print('###')
    print()

Fold:  1


KNN
RMSE: 0.5314010959371414
MAE: 0.35438377821174055
Random Forest
RMSE: 0.3945630206053723
MAE: 0.24052186695144256
Gradient Boosting
RMSE: 0.48857338612970036
MAE: 0.4025385563423702
###

Fold:  2
KNN
RMSE: 0.5597657004897894
MAE: 0.3480100746627986
Random Forest
RMSE: 0.39128875614481323
MAE: 0.23278735737038456
Gradient Boosting
RMSE: 0.5185777786699556
MAE: 0.38757856350394915
###

Fold:  3
KNN
RMSE: 0.457579085518153
MAE: 0.3121257343361092
Random Forest
RMSE: 0.3406208988031449
MAE: 0.2515308732974232
Gradient Boosting
RMSE: 0.397042306915877
MAE: 0.31708882339475625
###

Fold:  4
KNN
RMSE: 0.49989754610910836
MAE: 0.33280603633795225
Random Forest
RMSE: 0.42292959961459065
MAE: 0.24052931726242952
Gradient Boosting
RMSE: 0.4442560899852136
MAE: 0.3027907390695932
###

Fold:  5
KNN
RMSE: 0.6139571497689932
MAE: 0.3788904901073281
Random Forest
RMSE: 0.512186427659454
MAE: 0.26664929954530775
Gradient Boosting
RMSE: 0.6090369790663201
MAE: 0.42842308303331683
###



In [63]:
df_results = pd.DataFrame()
df_results['fold'] = cv_folds
df_results['regressor'] = reg_names
df_results['RMSE'] = cv_RMSE
df_results['MAE'] = cv_MAE
df_results['train_years'] = train_years
df_results['val_year'] = val_year
df_results

,fold,regressor,RMSE,MAE,train_years,val_year
0,1,KNN,0.531401,0.354384,"[2008, 2007, 2010, 2009]",[2011]
1,1,Random Forest,0.394563,0.240522,"[2008, 2007, 2010, 2009]",[2011]
2,1,Gradient Boosting,0.488573,0.402539,"[2008, 2007, 2010, 2009]",[2011]
3,2,KNN,0.559766,0.348010,"[2009, 2010, 2007, 2008, 2011]",[2012]
4,2,Random Forest,0.391289,0.232787,"[2009, 2010, 2007, 2008, 2011]",[2012]
5,2,Gradient Boosting,0.518578,0.387579,"[2009, 2010, 2007, 2008, 2011]",[2012]
6,3,KNN,0.457579,0.312126,"[2012, 2010, 2009, 2008, 2011, 2007]",[2013]
7,3,Random Forest,0.340621,0.251531,"[2012, 2010, 2009, 2008, 2011, 2007]",[2013]
8,3,Gradient Boosting,0.397042,0.317089,"[2012, 2010, 2009, 2008, 2011, 2007]",[2013]
9,4,KNN,0.499898,0.332806,"[2007, 2008, 2012, 2013, 2011, 2010, 2009]",[2014]


In [64]:
df_results.to_csv('../experiment_results/baseline_selection_results_Kea.csv',index=False)

In [65]:
df_results.groupby('regressor').agg({'RMSE': ['mean'], 'MAE': [ 'mean']})

,RMSE,MAE
,mean,mean
regressor,,
Gradient Boosting,0.491497,0.367684
KNN,0.532520,0.345243
Random Forest,0.412318,0.246404


In [66]:
## How to find from results mean RMSE for Random Forest 
mean_RMSE_random_forest = df_results[df_results['regressor'] == 'Random Forest'].agg({'RMSE': ['mean']}).values[0][0]
mean_MAE_random_forest = df_results[df_results['regressor'] == 'Random Forest'].agg({'MAE': ['mean']}).values[0][0]


In [67]:
print("As we can see, Random Forest has the best performance in both RMSE and MAE. \n\nMean RMSE is:\t {:.4f}".format(mean_RMSE_random_forest))
print("Mean MAE is:\t {:.4f}".format(mean_MAE_random_forest))


As we can see, Random Forest has the best performance in both RMSE and MAE. 

Mean RMSE is:	 0.4123
Mean MAE is:	 0.2464


## Refit Random Forest on train and predict on test set

In [71]:
print('The training set has years: ', list(df_train.Year.unique()))
print('The test set has years: ', list(test.Year.unique()))

The training set has years:  [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]
The test set has years:  [2016]


In [68]:
# Shuffle
train = df_train.sample(frac=1, random_state=SEED)
test = test.sample(frac=1, random_state=SEED)

# X and y
X_train = train.drop(columns=['Y_maize_major','Year'], axis=1)
y_train = train['Y_maize_major']
X_test = test.drop(columns=['Y_maize_major','Year'], axis=1)
y_test = test['Y_maize_major']

# Scale to [0,1] range
sc = MinMaxScaler()
X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(sc.transform(X_test), columns=X_test.columns)

In [69]:
# Fit on train and predict on test
reg = regressors['Random Forest']
reg.fit(X_train, y_train)
y_test_pred = reg.predict(X_test)

In [70]:
# Calculate and print metrics
rmse = mean_squared_error(y_test, y_test_pred, squared=False)
mae = mean_absolute_error(y_test, y_test_pred)
print('Performance on test set (year 2016) (baseline results)')
print('RMSE:',round(rmse,4))
print('MAE:',round(mae,4))

Performance on test set (year 2016) (baseline results)
RMSE: 0.3497
MAE: 0.2056
